In [ ]:
from itertools import combinations

# Simple Dataset
transactions = [['milk', 'bread', 'eggs'],
                ['milk', 'bread'],
                ['milk', 'eggs'],
                ['bread', 'eggs'],
                ['milk', 'bread', 'eggs']
]
def get_support(itemset, transactions):
  """Count how many transactions contain the itemset"""
  return sum(1 for t in transactions if set(itemset).issubset(t))

def generate_candidates(prev_freq, k):
  """Generate candidates of itemset of size k."""
  candidates = []
  items = list(prev_freq.keys())
  for i in range(len(items)):
    for j in range (i+1, len(items)):
      union = items[i].union(items[j])
      if len(union) == k and union not in candidates:
        candidates.append(union)
  return candidates # Moved return outside the loops

def apriori(transactions, min_support):
  """Simplified Apriori Algorithm."""
  # Step 1: 1-Itemsets
  items = set(i for t in transactions for i in t)
  freq = {frozenset([i]): get_support([i], transactions) for i in items}
  freq = {k: v for k, v in freq.items() if v >= min_support}
  all_frequent = [freq]

  k = 2
  while freq:
    candidates = generate_candidates(freq, k)
    freq = {c: get_support(c, transactions) for c in candidates}
    freq = {k_: v for k_, v in freq.items() if v >= min_support}
    if freq:
      all_frequent.append(freq)
      k += 1
  return all_frequent

def generate_rules(frequent_itemsets, min_confidence, transactions):
  """Generate association rules from frequent itemsets."""
  rules = []
  for level in frequent_itemsets[1:]: # Start from 2-Itemsets
    for itemset, support in level.items():
      for i in range(1, len(itemset)):
        for left in combinations(itemset, i):
          left = frozenset(left)
          right = itemset - left
          support_itemset = support / len(transactions)
          support_left = get_support(left, transactions) / len(transactions)
          confidence = support_itemset / support_left
          if confidence >= min_confidence:
            rules.append((set(left), set(right), support_itemset, confidence))
  return rules

# Run Apriori
min_support = 2
frequent_itemsets = apriori(transactions, min_support)

print("Frequent Itemsets:")
for level in frequent_itemsets:
  for itemset, support in level.items():
    print(set(itemset), "->", support)

# Generate Rules
rules = generate_rules(frequent_itemsets, min_confidence=0.6, transactions=transactions)
print("\nAssociation Rules:")
for left, right, sup, conf in rules:
  print(f"{left} -> {right} (support = {sup:.2f}, confidence = {conf:.2f})")

Frequent Itemsets:
{'eggs'} -> 4
{'bread'} -> 4
{'milk'} -> 4
{'eggs', 'bread'} -> 3
{'eggs', 'milk'} -> 3
{'bread', 'milk'} -> 3
{'eggs', 'bread', 'milk'} -> 2

Association Rules:
{'eggs'} -> {'bread'} (support = 0.60, confidence = 0.75)
{'bread'} -> {'eggs'} (support = 0.60, confidence = 0.75)
{'eggs'} -> {'milk'} (support = 0.60, confidence = 0.75)
{'milk'} -> {'eggs'} (support = 0.60, confidence = 0.75)
{'bread'} -> {'milk'} (support = 0.60, confidence = 0.75)
{'milk'} -> {'bread'} (support = 0.60, confidence = 0.75)
{'eggs', 'bread'} -> {'milk'} (support = 0.40, confidence = 0.67)
{'eggs', 'milk'} -> {'bread'} (support = 0.40, confidence = 0.67)
{'bread', 'milk'} -> {'eggs'} (support = 0.40, confidence = 0.67)
